<b><h3> Análisis Exploratorio Bivariado </h3></b>

Como se mencionó anteriormente, primero filtraremos por el idioma español y veremos los juegos más stremeados para este grupo:

In [ ]:
data_twitch_es <- data_twitch %>%
  filter(language == "Spanish")

In [ ]:
most_streamed_games_es <- data_twitch_es %>%
  group_by(most_streamed_game) %>%
    summarise(count = n()) %>%
     arrange(desc(count)) %>%
       mutate(percentage = (count / sum(count)) * 100)

In [ ]:
print(most_streamed_games_es)

In [ ]:
library(RColorBrewer)

colors_purples <- rep(brewer.pal(9, "Purples"), length.out = 27)

ggplot(most_streamed_games_es, aes(x = reorder(most_streamed_game, -count), 
                                   y = count, fill = most_streamed_game)) +
  geom_bar(stat = "identity", alpha = 0.8, color = "black") +
  geom_text(aes(label = paste0(round(percentage, 1), "%")), vjust = -0.5, size = 3.5) +
  labs(title = "Juegos más streameados en español",
       x = "Juego",
       y = "Cantidad de streamers") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  theme(legend.position = "none") +
  scale_fill_manual(values = colors_purples)

A diferencia de los juegos más jugados en la base de datos completa, vemos que en la comunidad de habla hispana cambia un poco. Sin embargo, la categoría de **Just Chatting** sigue siendo el primer lugar, por lo que podemos deducir que a la gente también le llama la atención actividades no sólo relacionadas a los juegos.

A continuación analizaremos la variable de **seguidores ganados por stream** en los juegos más stremeados de la comunidad hispana, de tal forma de ver cuales juegos generan un mayor **engagement** y así tener esta información en consideración.

In [ ]:
followers_per_game_es <- data_twitch_es %>%
                          group_by(most_streamed_game) %>%
                          summarise(mean_followers = mean(followers_gained_per_stream, na.rm = TRUE),
                          count = n()) %>%
                          arrange(desc(mean_followers))

In [ ]:
print(followers_per_game_es)

In [ ]:
ggplot(followers_per_game_es, aes(x = reorder(most_streamed_game, -mean_followers), 
                                  y = mean_followers, fill = mean_followers)) +
  geom_bar(stat = "identity", alpha = 0.8, color = "black") +
  labs(title = "Media de seguidores ganados por juego en español",
       x = "Juego más streameado",
       y = "Media de seguidores ganados por stream") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  theme(legend.position = "none") +
  scale_fill_gradient(low = "#E0E0F7", high = "#4A148C")

Este gráfico nos muestra la media de seguidores ganados por juego, lo que nos da una idea del nivel promedio de **engagement** para cada juego, sin embargo, esta media puede estar influenciada por valores atípicos, pocas observaciones como lo es en el los casos de categorías como **Music, FIFA 21, Warframe** y aquellos que tienen solo una observación, etc. Para esto analizamos un boxplot:

In [ ]:
followers_per_game_es_full <- data_twitch_es %>%
  select(most_streamed_game, followers_gained_per_stream)

In [ ]:
print(followers_per_game_es_full)

In [ ]:
ggplot(followers_per_game_es_full, aes(x = reorder(most_streamed_game, -followers_gained_per_stream, FUN = median), 
                                       y = followers_gained_per_stream)) +
  geom_boxplot(fill = "purple") +
  labs(title = "Distribución de seguidores ganados por juego en español",
       x = "Juego más streameado",
       y = "Seguidores ganados por stream") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

Teniendo en consideración los gráficos anteriores, descartaremos o diremos que aquellos juegos con sólo una observación serán consideradas opciones secundarias. Dado esto, podemos ver que juegos como **Just Chatting, Grand Theft Auto V, Minecraft o League of Legends** son opciones interesantes a profundizar. 

Para el caso de **Minecraft**, que es el con mayor media de seguidores ganados, vemos que la mediana se encuentra en la parte inferior del boxplot. Esto significa que la mayoría de los valores del IQR se encuentran concentrados en la parte inferior y la parte superior del IQR es más dispersa. En resumen, aunque la mayoría de los streamers tienen resultados "bajos" (mediana baja), algunos logran valores significativamente mayores. Este juego podría ser atractivo como una **oportunidad**, pero mi amigo debe aprender de los streamers que logran resultados altos para generar un mayor engagement.

Por otro lado, en el caso de **Grand Theft Auto V** la mayoría de streamers suele ganar una cantidad considerable de seguidores ya que la mediana se encuentra en la parte superior del IQR. Sin embargo, el hecho de que GTA V tenga una mediana alta y una caja larga significa que, en general, este juego genera un buen nivel de engagement, pero con una alta variabilidad entre los streamers.

Finalmente, **Just Chatting o League of Legends** tienen medianas similares y varias observaciones, por lo que en general los streamers de estos contenidos tienen una buena ganancia de seguidores y podrían ser opciones "seguras". Aunque hay diferencias en su variabilidad y existen algunos streamers que no generan buen engagement, podriamos decir que las estrategias en este contenido son claves.

In [ ]:
top_games_by_day_viewers <- data_twitch_es %>%
  group_by(most_active_day, most_streamed_game) %>%
  summarise(total_viewers = sum(avg_viewers_per_stream, na.rm = TRUE)) %>%
  arrange(most_active_day, desc(total_viewers)) %>%
  group_by(most_active_day) %>%
  slice_head(n = 5) 

top_games_by_day_viewers$most_active_day <- factor(top_games_by_day_viewers$most_active_day, levels = c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"))

custom_palette <- c("#8682bd", "#4c3f77", "#7c587f", "#a4bcbc", "#051126",
  "purple", "#007f97", "#f3f6e9", "#bcd8f9", "#5c87ba",
  "#6c5397", "#8d6cb1", "#4a2b7e", "#2e4277", "#abc4b8",
  "#f4b2c1", "#ad99d5", "#639eaf", "#95d0d1", "#3e5b82")

In [ ]:
ggplot(top_games_by_day_viewers, aes(x = most_active_day, y = total_viewers, fill = most_streamed_game)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Top 5 Juegos Más Jugados por Día con Más Espectadores",
       x = "Día de la Semana",
       y = "Total de Espectadores",
       fill = "Juego") +
  scale_fill_manual(values = custom_palette) +  # Aplicar la paleta personalizada
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
        legend.position = "bottom")

Ya para terminar el análisis de la variable catégorica relacionada a los juegos, con este gráfico más la información obtenida anteriormente, podríamos hacer un esquema de que juegos podría mi amigo stremear dependiendo del día e interéses del público.

Categorías como, **Just Chatting**, **League of Legends**, **Grand Theft Auto V** y **Minecraft** se repiten nuevamente, por lo que sin duda son las que principalmente agregaremos a nuestra recomendación.

In [ ]:
dias_mas_activos_es <- data_twitch_es %>%
  count(most_active_day) %>%
  arrange(desc(n)) %>% 
  mutate(percentage = n / sum(n) * 100)

In [ ]:
ggplot(dias_mas_activos_es, aes(x = reorder(most_active_day, -n), y = n)) +
  geom_bar(stat = "identity", fill = "#4A148C", color = "black") +
  geom_text(aes(label = paste0(round(percentage, 1), "%")), 
            vjust = -0.5,  
            size = 3.5) + 
  labs(title = "Días más activos en la Comunidad Hispana",
       x = "Día",
       y = "Cantidad de Streamers") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

Adicionalmente, vemos que los días más activos son los **Martes, Miércoles y Domingo**, a diferencia de cuando se consideraba el total de la base que era los **Martes, Miércoles y Jueves**. Esto, se refiere directamente a los días que los **Streamers** de habla hispana se encuentran más activos.

A continuación, analizaremos los días que hay mayor actividad por parte de los **Viewers**:

In [ ]:
views_por_dia <- data_twitch_es %>%
  group_by(most_active_day) %>%
  summarise(total_views = sum(total_views, na.rm = TRUE)) %>%
  arrange(desc(total_views))

views_por_dia <- views_por_dia %>%
  mutate(total_views_millions = total_views / 1e6)

In [ ]:
print(views_por_dia)

In [ ]:
ggplot(views_por_dia, aes(x = reorder(most_active_day, -total_views), 
                          y = total_views_millions)) +
  geom_bar(stat = "identity", fill = "#4A148C", color = "black") +
  geom_text(aes(label = paste0(round(total_views_millions, 2), "M")),  # Etiquetas en millones
            vjust = -0.5, 
            size = 3.5) + 
  labs(title = "Días con Más Views en la Comunidad Hispana",
       x = "Día",
       y = "Total de Views (en millones)") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
dias_mas_seguidores_es <- data_twitch_es %>%
  count(day_with_most_followers_gained) %>%
  arrange(desc(n))

In [ ]:
head(dias_mas_seguidores_es, 7)

In [ ]:
ggplot(dias_mas_seguidores_es, aes(x = reorder(day_with_most_followers_gained, -n), 
                                y = n)) +
  geom_bar(stat = "identity", fill = "#4A148C", color = "black") +
  geom_text(aes(label = n), 
            vjust = -0.5, 
            size = 3.5) + 
  labs(title = "Días con Mayores Seguidores Ganados en la Comunidad Hispana",
       x = "Día",
       y = "Número de Streams") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

Por otra parte, pero de la mano con lo anterior, los días con mas **Viewers o Espectadores** son los **Miércoles**. Y los días con mayores ocurrencias en máximos de seguidores ganados son los **Domingos**. Aunque quizás los días con mayores seguidores ganados no nos dan mucha información, ya que estos son valores únicos por cada Streamer y este puede depender de muchos factores. Lo que si resulta algo llamativo, es que casi el 50% de los streamers haya conseguido su máximo de seguidores en un stream, un día **Domingo o Lunes**.

In [ ]:
merged_data <- views_por_dia %>%
  left_join(dias_mas_activos_es, by = c("most_active_day" = "most_active_day")) %>%
  left_join(dias_mas_seguidores_es, by = c("most_active_day" = "day_with_most_followers_gained"))

merged_data <- merged_data %>%
  rename(
    total_views = total_views,  
    streamers = n.x,           
    followers_gained = n.y)

In [ ]:
merged_data_filtered <- merged_data %>%
  select(most_active_day, total_views, streamers, followers_gained)

merged_data_long <- merged_data_filtered %>%
  pivot_longer(cols = c(total_views, streamers, followers_gained), 
               names_to = "metric", 
               values_to = "value")

merged_data_long <- merged_data_long %>%
  group_by(metric) %>%
  mutate(value_normalized = (value - min(value)) / (max(value) - min(value))) %>%
  ungroup()

head(merged_data_long)

In [ ]:
ggplot(merged_data_long, aes(x = most_active_day, y = value_normalized, fill = metric)) +
  geom_bar(stat = "identity", position = "dodge") +  # Se usa dodge para separar las barras
  labs(title = "Análisis Comparativo: Días Más Activos, Views y Seguidores",
       x = "Día de la Semana",
       y = "Valor Normalizado",
       fill = "Métrica") +
  scale_fill_manual(values = c("total_views" = "#051126", "streamers" = "purple", "followers_gained" = "#c6bed8")) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) 

Según mi parecer acá las variables que más importan son los días que más stremean los creadores de contenido (Streamers), y los días con mayores vistas (Viewers). Como recomendación a mi amigo le diría de enfocar su energía los días **Miércoles, Jueves y Domingo**.

Por otra parte, y ya habiendo realizado un análisis bivariado entre las variables categóricas más relevantes, realizaremos un análisis de las variables numéricas mediante una matriz de confusión. Esto, con el objetivo de ver la correlación entre las distintas variables y comprobar si nuestras hipótesis planteadas anteriormente son o no respaldadas con evidencia.

In [ ]:
variables_numericas <- data_twitch %>%
  select_if(is.numeric)

In [ ]:
correlation_matrix <- cor(variables_numericas, use = "complete.obs")

In [ ]:
corrplot::corrplot(correlation_matrix, 
         method = "color",          # Método de visualización
         col = colorRampPalette(c("red", "white", "purple"))(200),  # Paleta de colores
         type = "lower",            # Muestra solo la parte inferior de la matriz
         order = "hclust",          # Ordena las variables según jerarquía
         tl.cex = 0.8,              # Tamaño de las etiquetas de las variables
         tl.col = "black",          # Color de las etiquetas
         cl.cex = 0.8,              # Tamaño de la leyenda
         title = "Matriz de Correlación", # Título del gráfico
         mar = c(0, 0, 1, 0),       # Ajuste de márgenes
         addCoef.col = "black",     # Agregar los valores de correlación en negro
         number.cex = 0.7,          # Ajusta el tamaño del texto de los números
         diag = FALSE)   

De la matriz de correlación podemos ver que **NO** hay variables numéricas con correlación alta. Sin embargo, variables en un rango de **0.5 a 0.7** serían consideradas **moderadas** y es donde pondremos enfoque.

Correlaciones tales como:

- **total_views** con **avg_viewers_per_stream:** 0.57
- **total_views** con **total_followers:** 0.65
- **active_days_per_week** con **total_time_streamed:** 0.57
- **total_games_streamed** con **avg_games_per_stream:** 0.54


In [ ]:
ggplot(data_twitch_es, aes(x = total_views, y = avg_viewers_per_stream)) +
  geom_point(aes(color = avg_viewers_per_stream), size = 3, alpha = 0.7) +
  scale_color_viridis_c(option = "magma") + # Escala de colores atractiva
  geom_smooth(method = "lm", color = "blue", linetype = "dashed", size = 1) +
  labs(title = "Relación entre Vistas Totales y Promedio de Espectadores por Stream",
       x = "Vistas Totales",
       y = "Promedio de Espectadores por Stream") +
  theme_minimal() +
  theme(plot.title = element_text(size = 15, face = "bold", hjust = 0.5),
        plot.caption = element_text(size = 8, hjust = 1),
        axis.title.x = element_text(margin = margin(t = 20)),
        axis.title.y = element_text(margin = margin(r = 20)))

In [ ]:
ggplot(data_twitch_es, aes(x = total_views, y = total_followers)) +
  geom_point(aes(color = total_followers), size = 3, alpha = 0.7) +
  scale_color_viridis_c(option = "magma") +
  geom_smooth(method = "lm", color = "blue", linetype = "dashed", size = 1) +
  labs(title = "Relación entre Vistas Totales y Seguidores Totales",
       x = "Vistas Totales",
       y = "Seguidores Totales") +
  theme_minimal() +
  theme(plot.title = element_text(size = 15, face = "bold", hjust = 0.5),
        plot.caption = element_text(size = 8, hjust = 1),
        axis.title.x = element_text(margin = margin(t = 20)),
        axis.title.y = element_text(margin = margin(r = 20)))

Estos gráficos analizan la correlación de la variable Vistas Totales (total_views) con el Promedio de Espectadores por Stream y Seguidores Totales, respectivamente. 

En ambos, a medida que aumenta el total de views, el promedio de espectadores por stream y seguidores totales también tiende a aumentar, aunque no de manera perfecta. Esto tiene sentido, ya que los streams con más vistas probablemente atraen más espectadores por transmisión, lo que se ve reflejado en el promedio de espectadores y seguidores.

In [ ]:
ggplot(data_twitch_es, aes(x = active_days_per_week, y = total_time_streamed)) +
  geom_point(aes(color = total_time_streamed), size = 3, alpha = 0.7) +
  scale_color_viridis_c(option = "magma") +
  geom_smooth(method = "lm", color = "blue", linetype = "dashed", size = 1) +
  labs(title = "Relación entre Días Activos por Semana y Tiempo Total de Stream",
       x = "Días Activos por Semana",
       y = "Tiempo Total de Stream") +
  theme_minimal() +
  theme(plot.title = element_text(size = 15, face = "bold", hjust = 0.5),
        plot.caption = element_text(size = 8, hjust = 1),
        axis.title.x = element_text(margin = margin(t = 20)), 
        axis.title.y = element_text(margin = margin(r = 20)))

A diferencia de los gráficos anteriores, acá podemos ver una distribución algo más "simétrica" con una pequeña línea de tendencia creciente. Es posible que los valores más bajos de **active_days_per_week** (como 1 o 2 días) correspondan a **total_time_streamed** más bajos (usuarios que transmiten menos tiempo). A medida que active_days_per_week aumenta, total_time_streamed también tiende a aumentar.

Además, vemos que existe una mayor concentración en rangos moderados de días activos (como 3-4 días por semana), y no muchos streamers están transmitiendo una cantidad de tiempo extremadamente alta (entre 6 y 7 días) por diversas razones tales como fatiga, cansancio, etc.

In [ ]:
ggplot(data_twitch_es, aes(x = total_games_streamed, y = avg_games_per_stream)) +
  geom_point(aes(color = avg_games_per_stream), size = 3, alpha = 0.7) +
  scale_color_viridis_c(option = "magma") +
  geom_smooth(method = "lm", color = "blue", linetype = "dashed", size = 1) +
  labs(title = "Relación entre Total de Juegos Transmitidos y Promedio de Juegos por Stream",
       x = "Total de Juegos Transmitidos",
       y = "Promedio de Juegos por Stream") +
  theme_minimal() + 
  theme(plot.title = element_text(size = 15, face = "bold", hjust = 0.5), 
        plot.caption = element_text(size = 8, hjust = 1), 
        axis.title.x = element_text(margin = margin(t = 20)), 
        axis.title.y = element_text(margin = margin(r = 20)))

Podemos ver una concentración en la parte izquierda e inferior/media del gráfico. En la mayoría de los casos, los streamers tienden a tener un total relativamente "bajo" de juegos transmitidos, sin embargo, de igual manera se ve una tendencia positiva que podría explicarse por su correlación moderada de 0,54.

Esto se puede interpretar como una mezcla de streamers que se especializan en pocos juegos (y por lo tanto, tienen un número bajo de juegos por stream) y otros que juegan una amplia variedad de juegos, pero en sesiones que pueden ser cortas o limitadas.

In [ ]:
summary(data_twitch_es$average_stream_duration)

A pesar que la variable **avg_stream_duration** no presenta una correlación fuerte con otras variables, creo que es importante tener en consideración, ya que una decisión clave a la hora de stremear. A mi parecer, como mi amigo está comenzando, debería stremear en rangos cercanos a la mediana o media, es decir, **entre 4 y 5 horas**.

<b><h3> Conclusiones </h3></b>

El objetivo principal de este proyecto fue analizar datos de streamings para ayudar a mi amigo a incrementar su reconocimiento como streamer en la plataforma. Para lograrlo, trabajamos con datos de los canales más reconocidos, y posteriormente filtramos para la comunidad en español, ya que es el idioma de preferencia de mi amigo. Enfocándonos en variables como juegos más populares, promedio de espectadores, duración de streams, entre otras. Utilizamos técnicas de análisis exploratorio y visualizaciones claras para identificar patrones significativos.

A continuación describiremos algunos hallazgos encontrados que serán finalmente, recomendaciones para mi amigo y los canales que vienen ya con una comunidad y quieren llevar su stream al siguiente nivel:

**1) Patrones en la Duración de Streams:**

La duración promedio de los streams no mostró una correlación significativa con otras variables como viewers o followers. Sin embargo, la mediana de duración más efectiva está alrededor **de 4 a 5 horas**, lo que parece ser un punto intermedio que balancea el tiempo de exposición sin perder la atención de los espectadores.

**2) Juegos Más Populares:**

Los juegos más transmitidos varían según el día de la semana. Por ejemplo, **"Just Chatting" y "League of Legends"** destacan consistentemente entre los más populares. Estos juegos atraen a una mayor cantidad de espectadores y podrían ser clave para construir una base de seguidores fieles.

Por otro lado, tenemos juegos como **Grand Theft Auto V** que tienen un buen engagement con el público y los datos muestran que es un juego que permite a los streamers obtener buenos seguidores y por ende, mayor exposición.

Finalmente, vemos que **Minecraft** es un juego que sería presentado como una **oportunidad** para destacar ya que tiene un buen potencial y existen streamers que consiguen muchos seguidores gracias a este juego. Es importante considerar las estrategias y copiarse de aquellos streamers que ya tienen éxito. 

Otros dos juegos que a mi parecer representan una **oportunidad**, serían el **FIFA 21** y **Valorant**, ya que si bien vemos que no tienen sufiiente data, son juegos que tienen una buena cantidad de seguidores obtenidos y están dentro del Top 5 más jugados. Creo que mi amigo podría agregarlos a su lista como opciones secundarias.

**3) Días de mayor actividad:**

**Miércoles, Jueves y Domingo** destacan como los días con mayor cantidad de viewers promedio, sugiriendo que estos días son estratégicos para transmitir y captar más público. Mi recomendación sería dependiendo del tiempo disponible de mi amigo, sin embargo, **días entre Martes a Jueves y Domingo (5 días)** son días estratégicos en los que podría ver mejores resultados a mediano plazo.

**4) Relación con los Viewers:**

Vimos como variables como **total_views** y **total_followers** están moderadamente correlacionadas, indicando que el crecimiento en vistas impulsa el aumento de seguidores. Esto es clave, ya que, si mi amigo tiene una buena estrategia de visibilidad podría incrementar considerablemente su cantidad de seguidores. Esrategias como dar visibilidad a su canal mediante clips o videos cortos en **Tik Tok, Youtube, u otras plataformas**, podrían ayudarlo mucho a crecer.

Las transmisiones más exitosas combinan consistencia en los horarios, elección de juegos populares y una interacción activa con la comunidad.

**5) Diversificación:**

De la mano con el punto anterior, crear contenido para otras plataformas podría dar visibilidad considerable, sin embargo, mi amigo también podría explorar juegos menos saturados pero con comunidades fieles, alternando con los más populares para captar nuevas audiencias.

Es importante monitorear métricas clave mensualmente para ajustar la estrategia según el rendimiento y tendencias actuales.

<b><h3> Reflexión Final </h3></b>

El análisis realizado demuestra que un enfoque basado en datos puede ser invaluable para mejorar las posibilidades de éxito como streamer. Mi amigo ahora cuenta con una hoja de ruta clara para optimizar su contenido y aumentar su reconocimiento, destacando la importancia de la consistencia, la elección estratégica de juegos y la conexión con su comunidad. **¡Con estas recomendaciones, está más cerca de alcanzar su meta de ser un streamer destacado!**

